# 라이브러리 불러오기

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='NanumBarunGothic')

# 데이터 불러오기

In [ ]:
data = pd.read_csv('/home/aiffeldefinition/해커톤/data/수요예측_data.csv', encoding = 'cp949')
data.head()

In [ ]:
df = data.copy()
df.head()

In [ ]:
target_group

# 데이터 전처리

In [ ]:
same_list = []

for idx, rows in target_group.iterrows():
  if rows['컬럼4'] >= rows['컬럼3']:
    same_list.append(idx)
  else:
    continue

target_group.drop(index=same_list, inplace=True)

In [ ]:
from datetime import datetime, timedelta
target_group['컬럼4'] = pd.DatetimeIndex(target_group['컬럼4'])
target_group['컬럼3'] = pd.DatetimeIndex(target_group['컬럼3'])

In [ ]:
target_group['컬럼20'] = target_group['컬럼4'] - target_group['컬럼4']
target_group['컬럼20'].describe()

In [ ]:
hours = []
for _, rows in target_group.iterrows():
  sc = rows['컬럼20'].total_seconds()
  hours.append(sc/3600)

target_group['컬럼15'] = hours

In [ ]:
# 소수 셋 째자리에서 반올림
target_group['컬럼15'] = target_group['컬럼15'].map(lambda x: round(x, 2))

### 이상치 제거

In [ ]:
target_group['컬럼15'].quantile(.99)

In [ ]:
target_group['컬럼15'].describe()

In [ ]:
target_group.drop(index = target_group[target_group['컬럼15'] > 52.79].index, inplace=True)

In [ ]:
target_group['컬럼15'].describe()

In [ ]:
target_group.컬럼7.value_counts()

In [ ]:
car_price_mean = car_price.groupby('분류').mean()
car_price_mean.reset_index(inplace=True)
car_price_mean

In [ ]:
car_price_mean.rename(columns = {'분류' : '컬럼7_'} ,inplace=True)
car_price_mean

In [ ]:
# 반올림
car_price_mean['컬럼8'] = car_price_mean['컬럼8'].map(lambda x: round(x, 2))
car_price_mean['컬럼9'] = car_price_mean['컬럼9'].map(lambda x: round(x, 2))

In [ ]:
target_group_price = pd.merge(target_group, car_price_mean, how='left', on='컬럼7_')
target_group_price.head()

In [ ]:
target_group_price.isnull().sum()

In [ ]:
def w_d(x):
  if x >= 5:
    return 1  
  else:
    return 0  

In [ ]:
target_group_price['컬럼17_'] = target_group_price['컬럼12'].map(lambda x : w_d(x))

In [ ]:
for idx, row in target_group_price.iterrows():
  if (row['컬럼12'] == 6) & (row['컬럼13'] >= 19):
    target_group_price.loc[idx, '컬럼17_'] = 0
  elif (row['컬럼12'] == 4) & (row['컬럼13'] >= 19):
    target_group_price.loc[idx, '컬럼17_'] = 1
  else:
    continue

In [ ]:
for idx, row in target_group_price.iterrows():
  if (row['컬럼10'] == 1) & (row['컬럼11'] == 1) :  
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 2) & (row['컬럼11'] == 4) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 2) & (row['컬럼11'] == 5) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 2) & (row['컬럼11'] == 6) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 3) & (row['컬럼11'] == 1) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 5) & (row['컬럼11'] == 6) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 6) & (row['컬럼11'] == 6) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 8) & (row['컬럼11'] == 15) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 9) & (row['컬럼11'] == 12) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 9) & (row['컬럼11'] == 13) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 10) & (row['컬럼11'] == 3) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  elif (row['컬럼10'] == 10) & (row['컬럼11'] == 9) : 
    target_group_price.loc[idx, '컬럼17_'] = 1
  else:
    continue

In [ ]:
target_group_price['컬럼16'] = None

for idx, row in target_group_price.iterrows():
  if row['컬럼17_'] == 0:
    target_group_price.loc[idx, '컬럼16'] = row['컬럼8'] * row['컬럼15']
  else:
    target_group_price.loc[idx, '컬럼16'] = row['컬럼9'] * row['컬럼15']

#### * 데이터프레임 저장

In [ ]:
target_group_price.to_pickle('/home/aiffeldefinition/해커톤/data/target_group_price_99del.pkl')

In [ ]:
target_group_price = pd.read_pickle('/home/aiffeldefinition/해커톤/data/target_group_price_99del.pkl')

In [ ]:
target_group_price.info()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import AgglomerativeClustering

In [ ]:
target_group_cluster = target_group_price.copy()

In [ ]:
target_group_cluster['컬럼6'].replace({'B':0, 'A':1}, inplace=True)
target_group_cluster['컬럼5'].replace({ 1: 0, 2 : 1}, inplace=True)

In [ ]:
car_onehot = pd.get_dummies(target_group_cluster['컬럼7_'])
month_onehot = pd.get_dummies(target_group_cluster['컬럼10'])
hour_onehot = pd.get_dummies(target_group_cluster['컬럼13'])

In [ ]:
target_group_cluster = pd.concat([target_group_cluster, car_onehot, month_onehot, hour_onehot], axis = 1)

In [ ]:
target_group_cluster.info()

# Scaling

In [ ]:
# Hierarchical Clustering과 kmeans는 거리기반 알고리즘이기 때문에 단위의 영향력을 제거하기 위해 표준화 진행
# (x-mean)/std로 변환하여 평균 0, 표준편차 1로 scaling을 맞춰주는 RobustScaler를 사용

#from sklearn.preprocessing import StandardScaler
#standard_scaler = StandardScaler()

#from sklearn.preprocessing import MinMaxScaler
#minmax_scaler = MinMaxScaler()

from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()

scale_cols = ['컬럼16', '컬럼8', '컬럼9', '컬럼15']
target_group_group_del = target_group_cluster.copy()

target_group_group_scaled_features = target_group_group_del[scale_cols]
target_group_group_del = target_group_cluster.drop(scale_cols, axis=1)

In [ ]:
target_group_group_scaled_features = pd.DataFrame(robust_scaler.fit_transform(target_group_group_scaled_features), columns= target_group_group_scaled_features.columns) # scaled된 데이터
target_group_group_scaled_features.index = target_group_group_del.index
target_group_group_scaled = pd.concat([target_group_group_del, target_group_group_scaled_features], axis=1)

In [ ]:
target_group_regions = target_group_cluster.groupby('지역').mean()
target_group_regions_scaled = target_group_group_scaled.groupby('지역').mean()

In [ ]:
target_group_regions

In [ ]:
target_group_regions_scaled

In [ ]:
regions = ['ADA', 'ACA', 'ADB', 'ABA', 'ADC', 'ACE', 'ACC',
           'AAA', 'ABB', 'ACB', 'AAB', 'ABD',
           'AAC', 'ABC', 'ACD', 'ABE']

weekend_ratio = []
for i in regions:
  df = target_group_cluster[target_group_cluster['지역'] == i]
  ratio = len(df[df['컬럼17_'] == 0]) / len(df['컬럼17_'])
  weekend_ratio.append(round(ratio, 4))

target_group_regions['컬럼21'] = weekend_ratio

In [ ]:
regions = ['ADA', 'ACA', 'ADB', 'ABA', 'ADC', 'ACE', 'ACC',
           'AAA', 'ABB', 'ACB', 'AAB', 'ABD',
           'AAC', 'ABC', 'ACD', 'ABE']

weekend_ratio = []
for i in regions:
  df = target_group_group_scaled[target_group_group_scaled['지역'] == i]
  ratio = len(df[df['컬럼17_'] == 0]) / len(df['컬럼17_'])
  weekend_ratio.append(round(ratio, 4))

target_group_regions_scaled['컬럼21'] = weekend_ratio

In [ ]:
target_group_regions

In [ ]:
drop_cols = ['컬럼14', '컬럼10', '컬럼11', '컬럼12', '컬럼13']
target_group_group_num = target_group_regions.drop(drop_cols, axis = 1)
target_group_group_num    

In [ ]:
drop_cols = ['컬럼14', '컬럼10', '컬럼11', '컬럼12', '컬럼13']
target_group_regions_scaled_num = target_group_regions_scaled.drop(drop_cols, axis = 1)
target_group_regions_scaled_num    

# Clustering - Hierarchical Clustering

In [ ]:
target_group_group_num.info()

In [ ]:
target_group_regions_scaled_num.info()

In [ ]:
# 왼쪽은 scaling 적용 X 데이터, 오른쪽은 scaling 적용 O 데이터
# 순서대로 single linkage, complete linkage, average linkage, centroid linkage, ward linkage 적용

linkage_list = ['single', 'complete', 'average', 'centroid', 'ward']
data = [target_group_group_num, target_group_regions_scaled_num]

fig, axes = plt.subplots(nrows=len(linkage_list), ncols=2, figsize=(16, 26))
for i in range(len(linkage_list)):
    for j in range(len(data)):
        hierarchical_single = linkage(data[j], method=linkage_list[i])
        dn = dendrogram(hierarchical_single, ax=axes[i][j])
        axes[i][j].title.set_text(linkage_list[i])
plt.show()

In [ ]:
# 위의 dendrogram 결과 중 ward가 비교적 균일하게 군집화가 이루어진 것 같아 
# cluster가 4가 되도록 클러스터링

agg_clustering = AgglomerativeClustering(n_clusters=4, linkage='ward')
agg_labels = agg_clustering.fit_predict(target_group_group_num)

In [ ]:
plt.figure(figsize=(20, 12))

plt.subplot(331)
sns.scatterplot(x='컬럼16', y='컬럼21', data=target_group_group_num, hue=agg_labels, palette='Set2')

plt.subplot(333)
sns.scatterplot(x='컬럼16', y='컬럼15', data=target_group_group_num, hue=agg_labels, palette='Set2')

plt.subplot(334)
sns.scatterplot(x='컬럼16', y= '컬럼5', data=target_group_group_num, hue=agg_labels, palette='Set2')

plt.subplot(336)
sns.scatterplot(x='컬럼21', y='컬럼5', data=target_group_group_num, hue=agg_labels, palette='Set2')

plt.subplot(337)
sns.scatterplot(x='컬럼21', y='컬럼15', data=target_group_group_num, hue=agg_labels, palette='Set2')

In [ ]:
agg_labels

In [ ]:
target_group_group_num.index

### Sclaed Data로 군집화

In [ ]:
# 2차원으로 시각화 해보았을 때는 scaling하기 전의 데이터보다 clustering 결과가 안 좋아보임.

agg_clustering = AgglomerativeClustering(n_clusters=4, linkage='ward')
labels = agg_clustering.fit_predict(target_group_regions_scaled_num)

plt.figure(figsize=(20, 12))
plt.subplot(331)
sns.scatterplot(x='컬럼16', y='컬럼21', data=target_group_regions_scaled_num, hue=labels, palette='Set2')

plt.subplot(332)
sns.scatterplot(x='컬럼16', y='컬럼17_', data=target_group_regions_scaled_num, hue=labels, palette='Set2')

plt.subplot(333)
sns.scatterplot(x='컬럼16', y='컬럼15', data=target_group_regions_scaled_num, hue=labels, palette='Set2')

plt.subplot(334)
sns.scatterplot(x='컬럼16', y= '컬럼5', data=target_group_regions_scaled_num, hue=labels, palette='Set2')

plt.subplot(336)
sns.scatterplot(x='컬럼21', y='컬럼5', data=target_group_regions_scaled_num, hue=labels, palette='Set2')

plt.subplot(337)
sns.scatterplot(x='컬럼21', y='컬럼15', data=target_group_regions_scaled_num, hue=labels, palette='Set2')

# Clustering - K-means

In [ ]:
# k에 따라 inertia_(군집 내 거리제곱합의 합)이 어떻게 변하는 지 시각화
def change_n_clusters(n_clusters, data):
    sum_of_squared_distance = []
    for n_cluster in n_clusters:
        kmeans = KMeans(n_clusters=n_cluster)
        kmeans.fit(target_group_group_num)
        sum_of_squared_distance.append(kmeans.inertia_)
        
    plt.figure(1 , figsize = (12, 6))
    plt.plot(n_clusters , sum_of_squared_distance , 'o')
    plt.plot(n_clusters , sum_of_squared_distance , '-' , alpha = 0.5)
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia')

In [ ]:
n_clusters = [2, 3, 4, 5, 6, 7, 8]

### not scaled data

In [ ]:
# 4가 elbow poin
change_n_clusters(n_clusters, target_group_group_num)

In [ ]:
# scaling하지 않은 데이터를 kmeans(k=4)로 학습하고 시각화
# 어느정도 군집화가 잘 된 것으로 보여짐

kmeans = KMeans(n_clusters=4)
kmeans.fit(target_group_group_num)

plt.figure(figsize=(20, 6))
plt.subplot(131)
sns.scatterplot(x='컬럼16', y='컬럼21', data=target_group_group_num, hue=kmeans.labels_,palette='coolwarm')

plt.subplot(132)
sns.scatterplot(x='컬럼16', y='컬럼17_', data=target_group_group_num, hue=kmeans.labels_, palette='coolwarm')

plt.subplot(133)
sns.scatterplot(x='컬럼16', y='컬럼15', data=target_group_group_num, hue=kmeans.labels_, palette='coolwarm')

In [ ]:
kmeans.labels_

In [ ]:
target_group_group_num.index

### scaled data

In [ ]:
# 데이터의 분포는 넓게 퍼진 것 같지만 군집별 구분이 잘 되지 않음

kmeans = KMeans(n_clusters=4)
kmeans.fit(target_group_regions_scaled_num)

plt.figure(figsize=(20, 6))
plt.subplot(131)
sns.scatterplot(x='컬럼16', y='컬럼21', data=target_group_regions_scaled_num, hue=kmeans.labels_,palette='coolwarm')

plt.subplot(132)
sns.scatterplot(x='컬럼16', y='컬럼17_', data=target_group_regions_scaled_num, hue=kmeans.labels_, palette='coolwarm')

plt.subplot(133)
sns.scatterplot(x='컬럼16', y='컬럼15', data=target_group_regions_scaled_num, hue=kmeans.labels_, palette='coolwarm')

# Clustering Evaluation

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import math

In [ ]:
### 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 실루엣 계수를 면적으로 시각화한 함수 작성
def visualize_silhouette(clustering, X_features, cluster_lists=[1]): 
    
    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        if clustering[0] == 'hierarchical':
            model = AgglomerativeClustering(n_clusters=n_cluster, linkage=clustering[1])
        elif clustering[0] == 'kmeans':
            model = KMeans(n_clusters=n_cluster)
        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        cluster_labels = model.fit_predict(X_features)
        
        #if clustering[0] == 'dbscan':
        #    n_cluster = len(set(cluster_labels))-1
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,4)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([-1, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")

## Hierarchical Clustering Evaluation

In [ ]:
params = {
    'linkage':['single', 'complete', 'average', 'ward'],
    'n_clusters':[2, 3, 4, 5, 6, 7, 8]}

scores_hierarcical= {}
for linkage in params['linkage']:
    for n_cluster in params['n_clusters']:
        hierarchical = AgglomerativeClustering(n_clusters=n_cluster, linkage=linkage)
        hierarchical.fit(target_group_group_num)
        score = silhouette_score(target_group_group_num, hierarchical.labels_)
        scores_hierarcical['_'.join(['original', linkage, str(n_cluster)])] = score
        
        hierarchical = AgglomerativeClustering(n_clusters=n_cluster, linkage=linkage)
        hierarchical.fit(target_group_regions_scaled_num)
        score = silhouette_score(target_group_regions_scaled_num, hierarchical.labels_)
        scores_hierarcical['_'.join(['scaling', linkage, str(n_cluster)])] = score

pd.Series(scores_hierarcical).sort_values(ascending=False)[:10]

# 전반적으로 scaling을 적용한 후에 silhouette score가 낮아짐
# 아무래도 scaling하기 전의 척도가 군집화를 수행함에 있어 적절했던 것으로 예상

In [ ]:
# silhouette score 확인
# scaled 데이터로 군집화했을 경우 매우 낮은 실루엣 지수

visualize_silhouette(['hierarchical', 'ward'], target_group_group_num, [3, 4, 5, 6, 7, 8]) # original data
visualize_silhouette(['hierarchical', 'ward'], target_group_regions_scaled_num, [3, 4, 5, 6, 7, 8]) # scaled data

# cluster의 수가 7일 때 실루엣계수가 가장 높지만, 모든 군집의 실루엣계수가 나타난 건 cluster의 수가 4일 때

## K-means Evaluation

In [ ]:
params = {'n_clusters':[2, 3, 4, 5, 6, 7, 8]}

scores_kmeans = {}
for n_cluster in params['n_clusters']:
    model = KMeans(n_clusters=n_cluster)
    model.fit(target_group_group_num)
    score = silhouette_score(target_group_group_num, model.labels_)
    scores_kmeans['_'.join(['original', str(n_cluster)])] = score
    
    model = KMeans(n_clusters=n_cluster)
    model.fit(target_group_regions_scaled_num)
    score = silhouette_score(target_group_regions_scaled_num, model.labels_)
    scores_kmeans['_'.join(['scaled', str(n_cluster)])] = score

pd.Series(scores_kmeans).sort_values(ascending=False)[:10]

# 계층적 군집화와 마찬가지로 전반적으로 scaling을 적용한 후에 silhouette score가 낮아짐

In [ ]:
visualize_silhouette(['kmeans'], target_group_group_num, [3, 4, 5, 6, 7, 8])
visualize_silhouette(['kmeans'], target_group_regions_scaled_num, [3, 4, 5, 6, 7, 8])

# 이번에도 cluster의 수가 7일 때 실루엣계수가 가장 높지만, 모든 군집의 실루엣계수가 나타난 건 cluster의 수가 4일 때

In [ ]:
cluster_0 = ['AAA', 'AAB', 'AAC']
cluster_1 = ['ABA', 'ABB', 'ABD', 'ABE', 'ABC']
cluster_2 = ['ACA', 'ACE', 'ACC', 'ACB', 'ACD']
cluster_3 = ['ADA', 'ADB', 'ADC']

def int_cluster(x):
  if x in cluster_0:
    return 0
  elif x in cluster_1:
    return 1
  elif x in cluster_2:
    return 2
  elif x in cluster_3:
    return 3

target_group_price['Cluster_num'] = target_group_price['지역'].map(lambda x : int_cluster(x))

In [ ]:
Clusters = pd.DataFrame(target_group_price.groupby('Cluster_num')[['컬럼16', '컬럼15']].mean())
Clusters.sort_values(by='컬럼16', ascending=False)

In [ ]:
target_group_price

# * target_group_cluster 데이터프레임 저장

In [ ]:
pd.to_pickle(target_group_price, '/home/aiffeldefinition/해커톤/data/target_group_preprocessed.pkl')